In [10]:
from pathlib import Path
import json
import os

from tinydb import TinyDB

current_dir = Path(os.getcwd()).absolute()
results_dir = current_dir.joinpath('results')
kv_data_dir = results_dir.joinpath('kvdb')
kv_data_dir.mkdir(parents=True, exist_ok=True)


class DocumentDB(object):
    def __init__(self, db_path):
        ## You can use the code from the previous exmaple if you would like
        people_json = kv_data_dir.joinpath('people.json')
        visited_json = kv_data_dir.joinpath('visited.json')
        sites_json = kv_data_dir.joinpath('sites.json')
        measurements_json = kv_data_dir.joinpath('measurements.json')

        self._db_path = Path(db_path)
        self._db = None
        ## TODO: Implement code
        
        # load json files
        with open(people_json) as p:
          people = json.load(p)
        
        with open(measurements_json) as m:
          measurements = json.load(m)
        
        with open(sites_json) as s:
          sites = json.load(s)
        
        with open(visited_json) as v:
          visits = json.load(v)
        
        # loop through people
        for person in people:
            # init a person 
            person_visits = []
            visit_site_id = None
            
            for visit in visits:
                visit_measurements = []
                
                # loop through measurements to see if measurement belongs to this person
                for measurement in measurements:
                    if (measurements[measurement]["person_id"] == people[person]["person_id"] 
                        and visits[visit]["visit_id"] == measurements[measurement]["visit_id"]):
                        
                            # This could get overwritten w/ each loop, but thats ok because
                            # each measurement for a visit should be at the same site
                            visit_site_id = visits[visit]["site_id"]
                            visit_measurements.append(measurements[measurement])
                    
                # add site to visit object, using site id from measurement loop
                if visit_site_id is not None:
                    visits[visit]["site"] = sites[visit_site_id]
                
                # add measurements to visit object
                if len(visit_measurements) > 0:
                    visits[visit]["measurements"] = visit_measurements
                
                    # add visit object to person visits
                    person_visits.append(visits[visit])
            
            # Add visit object to person object
            if len(person_visits) > 0:
                people[person]["visits"] = person_visits
            
            # load the db
            self._load_db(people[person])

        # close the db
        self._db.close()
        
    def _load_db(self, doc):
        self._db = TinyDB(self._db_path, indent=4, )
        ## TODO: Implement code
        # insert the document
        self._db.insert(doc)

In [11]:
db_path = results_dir.joinpath('patient-info.json')
if db_path.exists():
    os.remove(db_path)

db = DocumentDB(db_path)